In [42]:
import numpy as np
%run Grid_world.ipynb

In [43]:
SMALL_ENOUGH = 1e-3 # threshold for convergence

In [44]:
def print_values(V, g):
  for i in range(g.rows):
    print("---------------------------")
    for j in range(g.cols):
      v = V.get((i,j), 0)
      if v >= 0:
        print(" %.2f|" % v, end="")
      else:
        print("%.2f|" % v, end="") # -ve sign takes up an extra space
    print("")


def print_policy(P, g):
  for i in range(g.rows):
    print("---------------------------")
    for j in range(g.cols):
      a = P.get((i,j), ' ')
      print("  %s  |" % a, end="")
    print("")

In [45]:
def random_policy(states, grid, gamma):
    V = {}
    #Initialize zero value for all states
    for state in states:
        V[state] = 0.0
        
    run_nbr = 0
    while True:
        biggest_change = 0
        run_nbr += 1
        #print("Run:",run_nbr)
        
        for state in states:
            #print("Updating for state", state)
            old_v = V[state]
            #Make move only if its a valid one
            if state in grid.actions:
                grid.set_state(state)
                v_new = 0.0
                
                # Find the new value of this state
                p = 1.0/len(grid.actions[state])   #Assign equal probabilty to all actions
                for action in grid.actions[state]:
                    grid.set_state(state)
                    reward = grid.move(action)
                    v_new += p*(reward + gamma*V[grid.current_state()])  #Bellman Eq.
                
                V[state] = v_new
                biggest_change = max(biggest_change, np.abs(old_v - V[state]))
        
        if biggest_change < SMALL_ENOUGH:
            print("Random policy converged in run", run_nbr)
            break
    
    print("values for uniformly random actions:")
    print_values(V, grid)
    print("\n")

In [48]:
def fixed_policy(states, grid, gamma, policy):
    
    print("Policy Grid:")
    print_policy(policy, grid)
    
    V = {}
    #Initialize zero value for all states
    for state in states:
        V[state] = 0
        
    run_nbr = 0
    while True:
        biggest_change = 0
        run_nbr += 1
        #print("Run:",run_nbr)
        
        for state in states:
            #print("Updating for state", state)
            old_v = V[state]
            #Make move only if its a valid one
            if state in policy:
                grid.set_state(state)
                v_new = 0.0
                
                # Find the new value of this state
                p = 1.0   #Each state has only 1 action possible
                for action in policy[state]:
                    grid.set_state(state)
                    reward = grid.move(action)
                    v_new += p*(reward + gamma*V[grid.current_state()])  #Bellman Eq.
                
                V[state] = v_new
                biggest_change = max(biggest_change, np.abs(old_v - V[state]))
        
        if biggest_change < SMALL_ENOUGH:
            print("\nFixed Policy converged in run", run_nbr)
            break
    
    print("Values for Fixed Policy actions:")
    print_values(V, grid)
    print("\n")
    return V

In [49]:
if __name__ == '__main__':
    # iterative policy evaluation
    # given a policy, let's find it's value function V(s)
    # we will do this for both a uniform random policy and fixed policy
    # NOTE:
    # there are 2 sources of randomness
    # p(a|s) - deciding what action to take given the state
    # p(s',r|s,a) - the next state and reward given your action-state pair
    # we are only modeling p(a|s) = uniform
    # how would the code change if p(s',r|s,a) is not deterministic?

    biggest_change = 0
    
    grid = standard_grid()
    states = grid.all_states()
    gamma = 0.9
    random_policy(states, grid, gamma)
    
    policy = {
    (2, 0): 'U',
    (1, 0): 'U',
    (0, 0): 'R',
    (0, 1): 'R',
    (0, 2): 'R',
    (1, 2): 'R',
    (2, 1): 'R',
    (2, 2): 'R',
    (2, 3): 'U',
              }
    
    V = fixed_policy(states, grid, gamma, policy)

Random policy converged in run 10
values for uniformly random actions:
---------------------------
 0.06| 0.15| 0.27| 0.00|
---------------------------
-0.02| 0.00|-0.37| 0.00|
---------------------------
-0.11|-0.22|-0.38|-0.67|


Policy Grid:
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  R  |     |
---------------------------
  U  |  R  |  R  |  U  |

Fixed Policy converged in run 4
Values for Fixed Policy actions:
---------------------------
 0.81| 0.90| 1.00| 0.00|
---------------------------
 0.73| 0.00|-1.00| 0.00|
---------------------------
 0.66|-0.81|-0.90|-1.00|


